In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import dggs
from dggs import DGGS, io as dggs_io, uitools
import dggs.datasets as data
import dggs.applib

dg = DGGS()

In [ ]:
df, crs = data.act_nexis()
df = dggs.pd_compute_address(df, 15, crs=crs, dg=dg)
df.head()

In [ ]:
def mask_getter(scale_level):
    suburbs, crs = data.get_by_name('act-suburbs')
    
    def get(name, as_text = False):
        poly = suburbs.get(name)
        if poly is None:
            return None
        
        im = dggs.shape_to_mask(suburbs[name], crs=crs, scale_level=scale_level)
        if as_text:
            return dggs.mask_to_addresses(im)
        
        return im
    
    return get
        
mm = mask_getter(12)

query_addr = (mm('BRADDON', as_text=True) + 
              mm('TURNER', as_text=True))
query_addr[:4]+['..']+query_addr[-4:], len(query_addr)

In [ ]:
mm = dggs.applib.pd_naive_overlap(query_addr, df)

In [ ]:
df_ = df[mm]
df_[['value','people']].sum()

In [ ]:
cbr_map = dggs_io.h5_load('cbr-google-map.h5')
mm_im = dggs.mask_from_addresses(query_addr)

ax = uitools.DgDraw(plt.figure())

ax.imshow(cbr_map)
ax.imshow(mm_im, alpha=0.2, reset_axis=False, cmap='spring')
ax.scatter(df.addr, color='y', marker='o')
ax.scatter(df_.addr, color='c', marker='+')
ax.hide_axis()

In [ ]:
ax.zoom(mm_im.roi)
ax.figure